# 07wk-029: 체중감량(교호작용) / 회귀분석

최규빈  
2023-10-17

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-y6dmKt32J5hobALnT8wigT&si=ScK7ryQpemMS4cJd>

# 2. Imports

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn.linear_model 
import sklearn.tree
import sklearn.model_selection

# 3. Data

In [2]:
# n = 10000
# Supplement = np.random.choice([True, False], n)
# Exercise = np.random.choice([False, True], n)
# Weight_Loss = np.where(
#     (~Supplement & (~Exercise)),
#     np.random.normal(loc=0, scale=1, size=n),  
#     np.where(
#         (Supplement & (Exercise)),
#         np.random.normal(loc=15.00, scale=1, size=n),
#         np.where(
#             (~Supplement & (Exercise)),
#             np.random.normal(loc=5.00, scale=1, size=n),
#             np.random.normal(loc=0.5, scale=1, size=n)
#         )
#     )
# )
# df = pd.DataFrame({
#     'Supplement': Supplement,
#     'Exercise': Exercise,
#     'Weight_Loss': Weight_Loss
# })
df_train = pd.read_csv('https://raw.githubusercontent.com/guebin/MP2023/main/posts/weightloss.csv')

In [3]:
df_train

In [4]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss')

`-` 운동과 체중감량보조제를 병행하면 시너지가 나는 것 같음

# 4. 분석

`-` 분석1: 모형을 아래와 같이 본다. – 언더피팅

-   ${\bf X}$: `Supplement`, `Exercise`
-   ${\bf y}$: `Weight_Loss`

In [5]:
# step1
X = df_train[['Supplement','Exercise']]
y = df_train['Weight_Loss']
# step2 
predictr = sklearn.linear_model.LinearRegression()
# step3
predictr.fit(X,y)
# step4 
df_train['Weight_Loss_hat'] = predictr.predict(X)
#---#
print(f'train score = {predictr.score(X,y):.4f}')

train score = 0.8208

In [6]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss')

In [7]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss_hat')

-   운동을 하면 10키로 감량효과가 있다고 추정하고 있음.
-   보충제를 먹으면 5키로 감량효과가 있다고 추정하고 있음.
-   대충 (10,5)의 숫자를 바꿔가면서 적합해봤는데 이게 최선이라는 의미임

`-` 분석2: 모형을 아래와 같이 본다. – 딱 맞아요

-   ${\bf X}$: `Supplement`, `Exercise`, `Supplement` $\times$
    `Exercise`
-   ${\bf y}$: `Weight_Loss`

> Note: 기본적인 운동의 효과 및 보조제의 효과는 각각 `Supplement`,
> `Exercise` 로 적합하고 운동과 보조제의 시너지는
> `Supplement`$\times$`Exercise` 로 적합한다.

In [12]:
# step1 
X = df_train.eval('Interaction = Supplement * Exercise')[['Supplement','Exercise','Interaction']]
y = df_train['Weight_Loss']
# step2 
predictr = sklearn.linear_model.LinearRegression()
# step3 
predictr.fit(X,y)
# step4 -- pass 
df_train['Weight_Loss_hat'] = predictr.predict(X)
#---#
print(f'train score = {predictr.score(X,y):.4f}')

train score = 0.9728

In [13]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss')

In [14]:
df_train.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss_hat')

-   운동의 효과는 5정도 감량효과가 있다고 추정함.
-   보충제를 먹으면 0.5키로 감량효과가 있다고 추정함.
-   다만 운동을 하면서 보충제를 함께 먹을 경우 발생하는 추가적인
    시너지효과가 9.5정도라고 추정하는 것임.